In [1]:
# Task 1: YoY Q1 to end of sep for rewards, non-rewards
# Task 2: transaction percentile group movement

import pandas as pd
import os
import numpy as np
import datetime
import gc

In [2]:
1+2

3

In [3]:
rewards_sales_1=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/From_Sp/combinedtransactions_0811.csv",
                            dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_id','location_id','total_transaction_amt'])
rewards_sales_1['transaction_date']=rewards_sales_1['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
# rewards_sales_1=rewards_sales_1[rewards_sales_1['transaction_date']>=datetime.date(2017,2,1)]
rewards_sales_1=rewards_sales_1.drop_duplicates()
rewards_sales_1=rewards_sales_1[rewards_sales_1['location_id']!="6990"]

In [3]:
# QC rewards sales by day

rewards_sales_1_2017=rewards_sales_1[(rewards_sales_1['transaction_date']>=datetime.date(2017,3,1)) & (rewards_sales_1['transaction_date']<=datetime.date(2017,4,30))]
rewards_sales_1_2018=rewards_sales_1[(rewards_sales_1['transaction_date']>=datetime.date(2018,3,1)) & (rewards_sales_1['transaction_date']<=datetime.date(2018,4,30))]
rewards_sales_1_2017['total_transaction_amt']=rewards_sales_1_2017['total_transaction_amt'].astype(float)
rewards_sales_1_2018['total_transaction_amt']=rewards_sales_1_2018['total_transaction_amt'].astype(float)

In [4]:
rewards_sales_1_2017['total_transaction_amt'].max()

'999.99'

In [6]:
rewards_sales_1_2017['total_transaction_amt'].min()

-480986.28000000003

In [7]:
rewards_sales_1_2018['total_transaction_amt'].max()

8685.1800000000003

In [8]:
rewards_sales_1_2018['total_transaction_amt'].min()

-4968.7399999999998

In [4]:


qc_rewards_sales_1_2017=rewards_sales_1_2017.groupby(['location_id','transaction_date'])['total_transaction_amt'].sum().to_frame().reset_index()
qc_rewards_sales_1_2018=rewards_sales_1_2018.groupby(['location_id','transaction_date'])['total_transaction_amt'].sum().to_frame().reset_index()



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
qc_rewards_sales_1_2017_wide=qc_rewards_sales_1_2017.pivot(index="location_id",columns="transaction_date",values="total_transaction_amt")
qc_rewards_sales_1_2018_wide=qc_rewards_sales_1_2018.pivot(index="location_id",columns="transaction_date",values="total_transaction_amt")
qc_rewards_sales_1_2017_wide=qc_rewards_sales_1_2017_wide.reset_index()
qc_rewards_sales_1_2018_wide=qc_rewards_sales_1_2018_wide.reset_index()


In [6]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis/output_for_rewards_YoY/QC_by_store_by_day_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
qc_rewards_sales_1_2017_wide.to_excel(writer,"2017",index=False)
qc_rewards_sales_1_2018_wide.to_excel(writer,"2018",index=False)
writer.save()

In [7]:
len(rewards_sales_1['transaction_date'].unique())

556

In [4]:
rewards_sales_2=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180828-131144-584.txt",
                             dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'])
rewards_sales_2=rewards_sales_2.drop_duplicates()
rewards_sales_2=rewards_sales_2.rename(columns={"transaction_dt":"transaction_date"})
rewards_sales_2=rewards_sales_2[rewards_sales_2['location_id']!="6990"]
rewards_sales_2['transaction_date']=rewards_sales_2['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

rewards_sales_3=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180911-133008-590.txt",
                             dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'])
rewards_sales_3=rewards_sales_3.drop_duplicates()
rewards_sales_3=rewards_sales_3.rename(columns={"transaction_dt":"transaction_date"})
rewards_sales_3=rewards_sales_3[rewards_sales_3['location_id']!="6990"]
rewards_sales_3['transaction_date']=rewards_sales_3['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

rewards_sales_4=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180925-132958-543.txt",
                             dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','total_transaction_amt'])
rewards_sales_4=rewards_sales_4.drop_duplicates()
rewards_sales_4=rewards_sales_4.rename(columns={"transaction_dt":"transaction_date"})
rewards_sales_4=rewards_sales_4[rewards_sales_4['location_id']!="6990"]
rewards_sales_4['transaction_date']=rewards_sales_4['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

In [5]:
rewards_sales=rewards_sales_1.append(rewards_sales_2).append(rewards_sales_3).append(rewards_sales_4)
del rewards_sales_1
del rewards_sales_2
del rewards_sales_3
del rewards_sales_4
rewards_sales=rewards_sales.reset_index()
del rewards_sales['index']
rewards_sales=rewards_sales.drop_duplicates()
rewards_sales['total_transaction_amt']=rewards_sales['total_transaction_amt'].astype(float)

gc.collect()

49

In [8]:
2+3

5

In [6]:
exclude_transaction=rewards_sales[rewards_sales['total_transaction_amt']<=-400000]

In [14]:
exclude_transaction

,customer_id_hashed,transaction_date,transaction_id,location_id,total_transaction_amt
23266017,5efa406f90b87aee3110c8dcebfaddcc246364bc6817e8...,2018-05-12,3599,5191,-480985.06
77425834,fe6babefed844dc4914bf0f8d42c0a79f0d0f7150c35f3...,2017-03-13,9040,840,-480986.28


In [7]:
rewards_sales=rewards_sales[rewards_sales['total_transaction_amt']>-400000]

In [9]:
# QC rewards sales by day

QC_rewards_sales_by_day=rewards_sales.groupby(['location_id','transaction_date'])['total_transaction_amt'].sum().to_frame().reset_index()

QC_rewards_sales_by_day.shape


(1164303, 3)

In [10]:
QC_rewards_sales_by_day_non_location=QC_rewards_sales_by_day.groupby(['transaction_date'])['total_transaction_amt'].sum().to_frame().reset_index()


In [15]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis/output_for_rewards_YoY/QC_by_store_by_day_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
QC_rewards_sales_by_day.to_excel(writer,"by_day_by_store",index=False)
QC_rewards_sales_by_day_non_location.to_excel(writer,"by_day",index=False)
exclude_transaction.to_excel(writer,"exclude_transaction",index=False)
writer.save()

In [13]:
QC_rewards_sales_by_day.shape

(1164303, 3)

In [11]:
'''
rewards_sales.sort_values(['total_transaction_amt']).head(20)
'''

,customer_id_hashed,transaction_date,transaction_id,location_id,total_transaction_amt
42331974,fe6babefed844dc4914bf0f8d42c0a79f0d0f7150c35f3...,2017-03-13,9040,840,-480986.28
23266017,5efa406f90b87aee3110c8dcebfaddcc246364bc6817e8...,2018-05-12,3599,5191,-480985.06
80493862,2ba3493b2c07c70af61ad772a63f714d8d2a348aab3de5...,2017-11-17,5481,1952,-6283.86
80264310,77c6f463abb1b60087f51720da4ef14f12c3bf99f22fb8...,2017-11-18,2423,4528,-5399.87
12617830,c29c377c432774296cda2805e7d340d8bc41d4f863ce49...,2018-03-02,2001,1817,-4968.74
33064432,27f886ffb85fe7f4796f9b890216d982f40e34b98ac421...,2018-06-22,5974,5302,-4816.39
72396630,32bceeb909e4be16cfc129e15867a62b99103820d48d79...,2017-09-30,1679,4127,-4799.88
93140740,d2475cddfa5c2aa5b55c71b25408a9c1346fa94094e039...,2018-08-08,2131,536,-4648.89
33164482,1893373013a363271ccbb09010cdd00490bcd2337d874b...,2018-07-07,4649,603,-4178.89
23192565,14abe04be98148c1db4a066292a7e92591fb765511c2a1...,2018-04-22,2521,4126,-4074.26


In [11]:
def add_week_end_date(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y
rewards_sales['week_end_dt']=rewards_sales['transaction_date'].apply(lambda x: add_week_end_date(x))

In [27]:
rewards_sales_2018=rewards_sales[(rewards_sales['week_end_dt']>=datetime.date(2018,2,10)) & (rewards_sales['week_end_dt']<=datetime.date(2018,9,22))]


In [28]:

rewards_sales_2017=rewards_sales[(rewards_sales['week_end_dt']>=datetime.date(2017,2,11)) & (rewards_sales['week_end_dt']<=datetime.date(2017,9,23))]
rewards_sales_2017['total_transaction_amt']=rewards_sales_2017['total_transaction_amt'].astype(float)

# del rewards_sales
gc.collect()

rewards_sales_2017_by_store_by_week=rewards_sales_2017.groupby(['location_id','week_end_dt'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_sales_2018_by_store_by_week=rewards_sales_2018.groupby(['location_id','week_end_dt'])['total_transaction_amt'].sum().to_frame().reset_index()

rewards_sales_2017_by_store_by_week_wide=rewards_sales_2017_by_store_by_week.pivot(index="location_id",columns="week_end_dt",values="total_transaction_amt").reset_index()
rewards_sales_2018_by_store_by_week_wide=rewards_sales_2018_by_store_by_week.pivot(index="location_id",columns="week_end_dt",values="total_transaction_amt").reset_index()

writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis/output_for_rewards_YoY/BL_rewards_sales_YoY_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
rewards_sales_2017_by_store_by_week.to_excel(writer,"rewards_sales_2017_long",index=False)
rewards_sales_2018_by_store_by_week.to_excel(writer,"rewards_sales_2018_long",index=False)
rewards_sales_2017_by_store_by_week_wide.to_excel(writer,"rewards_sales_2017",index=False)
rewards_sales_2018_by_store_by_week_wide.to_excel(writer,"rewards_sales_2018",index=False)
writer.save()


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
rewards_sales_2017_by_store_by_week=rewards_sales_2017_by_store_by_week.rename(columns={"total_transaction_amt":'sales',"week_end_dt":"week_end_date"})
rewards_sales_2017_by_store_by_week['type']="rewards"
rewards_sales_2018_by_store_by_week=rewards_sales_2018_by_store_by_week.rename(columns={"total_transaction_amt":'sales',"week_end_dt":"week_end_date"})
rewards_sales_2018_by_store_by_week['type']="rewards"
weeks_included=rewards_sales_2017_by_store_by_week['week_end_date'].unique().tolist()+rewards_sales_2018_by_store_by_week['week_end_date'].unique().tolist()


In [30]:
len(weeks_included)

66

In [31]:
total_sales=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-09-29.csv",dtype=str)
total_sales=total_sales[['location_id','week_end_date','sales']]
total_sales['sales']=total_sales['sales'].astype(float)
total_sales['week_end_date']=total_sales['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
total_sales=total_sales[total_sales['week_end_date'].isin(weeks_included)]
total_sales=total_sales[total_sales['sales']>0]
total_sales['type']='total'
output_rewards=rewards_sales_2017_by_store_by_week.append(rewards_sales_2018_by_store_by_week).append(total_sales)


In [32]:
output_rewards.head(2)

,location_id,week_end_date,sales,type
0,1,2017-02-11,32053.57,rewards
1,1,2017-02-18,36331.73,rewards


In [33]:
inclusion_stores=output_rewards.groupby(['location_id'])['week_end_date'].count().to_frame().reset_index()
inclusion_stores['inclusion']=np.where(inclusion_stores['week_end_date']==132,"Inclusions","Exclusion")
inclusion_stores=inclusion_stores[['location_id','inclusion']]
output_rewards=pd.merge(output_rewards,inclusion_stores,on="location_id",how="left")


In [34]:
output_rewards_total=output_rewards[output_rewards['type']=='total'][['location_id','week_end_date','sales','inclusion']].rename(columns={"sales":"total_sales"})
output_rewards_rewards=output_rewards[output_rewards['type']=='rewards'][['location_id','week_end_date','sales','inclusion']].rename(columns={"sales":"rewards_sales"})

output_rewards_non_rewards=pd.merge(output_rewards_total,output_rewards_rewards,on=["location_id","week_end_date","inclusion"],how="outer").fillna(0)
output_rewards_non_rewards['non_rewards_sales']=output_rewards_non_rewards['total_sales']-output_rewards_non_rewards['rewards_sales']

del output_rewards_non_rewards['total_sales']
del output_rewards_non_rewards['rewards_sales']


output_rewards_total=output_rewards_total.rename(columns={"total_sales":"sales"})
output_rewards_non_rewards=output_rewards_non_rewards.rename(columns={"non_rewards_sales":"sales"})
output_rewards_rewards=output_rewards_rewards.rename(columns={"rewards_sales":"sales"})

output_rewards_rewards['type']="rewards"
output_rewards_non_rewards['type']="non_rewards"
output_rewards_total['type']="total"

output_rewards_to_Joann=output_rewards_total.append(output_rewards_rewards).append(output_rewards_non_rewards)

In [35]:
output_rewards_to_Joann=output_rewards_to_Joann[['location_id','week_end_date','type','inclusion','sales']]

In [36]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis'

In [37]:
output_rewards_to_Joann.to_csv("/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis/output_for_rewards_YoY/BL_sales_8_months_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [38]:
output_rewards_total.head(2)

,location_id,week_end_date,sales,inclusion,type
93751,1,2017-02-11,65529.31,Inclusions,total
93752,1,2017-02-18,72052.29,Inclusions,total


# 2018_July_1_Split

In [9]:
rewards_sales_2018_Apr_June=rewards_sales_2018[(rewards_sales_2018['transaction_date']>=datetime.date(2018,4,1)) & (rewards_sales_2018['transaction_date']<=datetime.date(2018,6,30))]
rewards_sales_2018_July_Sep=rewards_sales_2018[(rewards_sales_2018['transaction_date']>=datetime.date(2018,7,1)) & (rewards_sales_2018['transaction_date']<=datetime.date(2018,9,30))]



In [10]:
rewards_sales_2018_Apr_June_sales=rewards_sales_2018_Apr_June.groupby(["customer_id_hashed"])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_sales_2018_Apr_June=rewards_sales_2018_Apr_June[rewards_sales_2018_Apr_June['total_transaction_amt']>0]
rewards_sales_2018_Apr_June_trans=rewards_sales_2018_Apr_June.groupby(["customer_id_hashed"])['total_transaction_amt'].count().to_frame().reset_index()

rewards_sales_2018_July_Sep_sales=rewards_sales_2018_July_Sep.groupby(["customer_id_hashed"])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_sales_2018_July_Sep=rewards_sales_2018_July_Sep[rewards_sales_2018_July_Sep['total_transaction_amt']>0]
rewards_sales_2018_July_Sep_trans=rewards_sales_2018_July_Sep.groupby(["customer_id_hashed"])['total_transaction_amt'].count().to_frame().reset_index()


In [11]:
rewards_sales_2018_Apr_June_sales=rewards_sales_2018_Apr_June_sales.rename(columns={"total_transaction_amt":"sales_Apr_June"})
rewards_sales_2018_Apr_June_trans=rewards_sales_2018_Apr_June_trans.rename(columns={"total_transaction_amt":"trans_Apr_June"})

rewards_sales_2018_July_Sep_sales=rewards_sales_2018_July_Sep_sales.rename(columns={"total_transaction_amt":"sales_July_Sep"})
rewards_sales_2018_July_Sep_trans=rewards_sales_2018_July_Sep_trans.rename(columns={"total_transaction_amt":"trans_July_Sep"})




In [17]:
all_ids_df=pd.merge(rewards_sales_2018_Apr_June_sales,rewards_sales_2018_Apr_June_trans,on="customer_id_hashed",how="left")
all_ids_df=pd.merge(all_ids_df,rewards_sales_2018_July_Sep_sales,on="customer_id_hashed",how="left")
all_ids_df=pd.merge(all_ids_df,rewards_sales_2018_July_Sep_trans,on="customer_id_hashed",how="left")
all_ids_df=all_ids_df.fillna(0)

check_hist_by_id_trans_freq=rewards_sales_2018_Apr_June_trans.groupby(['trans_Apr_June'])['customer_id_hashed'].count().to_frame().reset_index()
check_hist_by_id_trans_freq.to_csv("/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis/check_hist_by_id_trans_freq.csv")

# To be continued
# Use 10 as the cut of (as the 99 percentile)


In [15]:
rewards_sales_2018_Apr_June_sales.shape

(7488118, 2)

In [13]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Loyal_members/Segment_Movement_analysis'

In [88]:
np.percentile(rewards_sales_2018_Apr_June_trans['trans_Apr_June'],98)

8.0

In [89]:
np.percentile(rewards_sales_2018_Apr_June_trans['trans_Apr_June'],99)

10.0

In [91]:
np.percentile(rewards_sales_2018_Apr_June_trans['trans_Apr_June'],99.5)

13.0

In [92]:
np.percentile(rewards_sales_2018_Apr_June_trans['trans_Apr_June'],97)

7.0

In [93]:
np.percentile(rewards_sales_2018_Apr_June_trans['trans_Apr_June'],97.5)

8.0

In [94]:
rewards_sales_2018_Apr_June_trans['trans_Apr_June'].max()

1518

In [95]:
np.percentile(rewards_sales_2018_Apr_June_trans['trans_Apr_June'],99.75)

16.0

In [96]:
np.percentile(rewards_sales_2018_Apr_June_trans['trans_Apr_June'],99.9)

22.0

In [ ]:
1+1